# setup

In [4]:
import ROOT

In [2]:
filepath = '/atlas/local/BtagOptimizationNtuples/group.perf-flavtag.410000.PowhegPythiaEvtGen.AOD.e3698_s2997_r8903_r8906.v21-1.db-b5223bf2_Akt4EMTo/group.perf-flavtag.11010668.Akt4EMTo._000001.root'

In [3]:
rootfile = ROOT.TFile(filepath)
roottree = rootfile.Get("bTagAntiKt4EMTopoJets")

In [ ]:
import numpy as np
from numpy.lib.recfunctions import stack_arrays
from root_numpy import root2array, root2rec
import glob

In [ ]:
data = root2array(filepath)

In [ ]:
print type(data) 
branches = data.dtype.names
print len(data.dtype.names) ##check branch size
print data.shape ##check file size

In [ ]:
data[['jet_pt', 'jet_eta', 'jet_phi', 'jet_m']][10]

In [ ]:
data = root2array(filepath, selection='njets>7')
data.shape

In [ ]:
import pandas as pd


In [ ]:
df = pd.DataFrame(data)

In [ ]:
def root2pandas(files_path, tree_name, **kwargs):
    '''
    Args:
    -----
        files_path: a string like './data/*.root', for example
        tree_name: a string like 'bTag_AntiKt4EMTopoJets' corresponding to the name of the folder inside the root 
                   file that we want to open
        kwargs: arguments taken by root2array, such as branches to consider, start, stop, step, etc
    Returns:
    --------    
        output_panda: a pandas dataframe like allbkg_df in which all the info from the root file will be stored
    
    Note:
    -----
        if you are working with .root files that contain different branches, you might have to mask your data
        in that case, return pd.DataFrame(ss.data)
    '''
    # -- create list of .root files to process
    files = glob.glob(files_path)
    
    # -- process ntuples into rec arrays
    ss = stack_arrays([root2array(fpath, tree_name, **kwargs).view(np.recarray) for fpath in files])

    try:
        return pd.DataFrame(ss)
    except Exception:
        return pd.DataFrame(ss.data)

In [ ]:
df = root2pandas(filepath,
           'bTag_AntiKt4EMTopoJets', stop=100)

In [ ]:
df.to_hdf('test_pd.h5', 'data')

In [ ]:
new_df = pd.read_hdf('test_pd.h5', 'data')

In [ ]:
new_df

In [ ]:
jf_df = df[[key for key in df.keys() if (key.startswith('jet_jf') and '_vtx_' not in key)]]


In [ ]:
jf_df.keys()


In [ ]:
def flatten(column):
    '''
    Args:
    -----
        column: a column of a pandas df whose entries are lists (or regular entries -- in which case nothing is done)
                e.g.: my_df['some_variable'] 

    Returns:
    --------    
        flattened out version of the column. 

        For example, it will turn:
        [1791, 2719, 1891]
        [1717, 1, 0, 171, 9181, 537, 12]
        [82, 11]
        ...
        into:
        1791, 2719, 1891, 1717, 1, 0, 171, 9181, 537, 12, 82, 11, ...
    '''
    try:
        return np.array([v for e in column for v in e])
    except (TypeError, ValueError):
        return column

In [ ]:
jf_df_flat = pd.DataFrame({k: flatten(c) for k, c in jf_df.iteritems()})

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
matplotlib.rcParams.update({'font.size': 16})
%matplotlib inline

In [ ]:
flavor = flatten(df['jet_LabDr_HadF'])
flavor_pids = np.unique(flavor)

In [ ]:
for key in jf_df_flat.keys(): # plot the various variables one by one on different graphs
    
    # set up your figures
    fig = plt.figure(figsize=(8, 6), dpi=100)
    # specify ranges and binning strategies that make sense
    bins = np.linspace(
        min(jf_df_flat[key][jf_df_flat[key]!= -99]), # min
        max(jf_df_flat[key]), # max
        50 # number of bins
    )
    # select your favorite matplotlib color palette
    color = iter(cm.hsv(np.linspace(0, 0.8, len(flavor_pids))))
    # plot the histogram for each flavor using a different color
    for k in flavor_pids:
        c = next(color)
        _ = plt.hist(jf_df_flat[key][flavor == k][jf_df_flat[key]!= -99], 
                    bins=bins, histtype='step', label='Flavor = {}'.format(k), color=c,
                    normed=True)
        
    # prettify your histograms
    plt.xlabel(key)
    plt.ylabel('Arbitrary Units')
    plt.legend()
    plt.show()


In [ ]:
X = jf_df_flat.as_matrix() # I think this is the same as jf_df_flat.values


In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
from sklearn.preprocessing import StandardScaler


In [ ]:
le = LabelEncoder()
y = le.fit_transform(flavor)

In [ ]:
y

In [ ]:
ix = range(X.shape[0]) # array of indices, just to keep track of them for safety reasons and future checks

In [ ]:
X_train, X_test, y_train, y_test, ix_train, ix_test = train_test_split(X, y, ix, train_size=0.8)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train

In [ ]:
jf_df.keys()

In [ ]:
from keras.datasets.boston_housing import load_data

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
from keras.layers import Dense, Input, Activation
from keras.models import Model
from keras.utils import plot_model

In [ ]:
# we define the input shape (i.e., how many input features) **without** the batch size
x = Input(shape=(22, ))

# all Keras Ops look like z = f(z) (like functional programming)
h = Dense(40)(x)
h = Activation('relu')(h)

h = Dense(40)(h)
h = Activation('relu')(h)

# our output is a single number, the house price.
y = Dense(1)(h)

# A model is a conta
net = Model(x, y)

In [ ]:
net.compile(optimizer='adam', loss='mse')

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
callbacks = [
    # if we don't have a decrease of the loss for 10 epochs, terminate training.
    EarlyStopping(verbose=True, patience=10, monitor='val_loss'), 
    # Always make sure that we're saving the model weights with the best val loss.
    ModelCheckpoint('model.h5', monitor='val_loss', verbose=True, save_best_only=True)]

In [ ]:
history = net.fit(X_train, y_train, validation_split=0.2, epochs=10, verbose=2, callbacks=callbacks)